![Forecasting UK Mortgage Rates](resources/images/banner.png)

# End-to-End Machine Learning Pipeline for Forecasting UK Mortgage Rates
## Overview
UK mortgage rates directly impact affordability, re-financing decisions, and lender risk management. The objective of this project is to build a machine learning pipeline that forecasts short-term mortgage rate movements to support decisions such as re-financing timing, product pricing, and risk assessment.

The project demonstrates a complete end-to-end workflow from data acquisition to model evaluation and can be converted into a production-ready pipeline.

## 1. Data Loading & Preprocessing
In this section, we'll import the relevant libraries and load CSVs, clean column names, convert dates and align monthly data, finalizing with merging all the dataframes into one so we can proceed with our analysis.

In [1]:
# Loading libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Loading the dataframes
fixed_2 = pd.read_csv('resources/data/2y_fixed.csv')
fixed_5 = pd.read_csv('resources/data/5y_fixed.csv')
base_rate = pd.read_csv('resources/data/BoE_base_rate.csv')
cpih = pd.read_csv('resources/data/Inflation(CPIH).csv')

# Confirming load
display(fixed_2.head())
display(fixed_5.head())
display(base_rate.head())
display(cpih.head(10))

,Date,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (95% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [a] [b] [c] IUM2WTL,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (90% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [d] [e] [b] [c] IUMB482,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (75% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [a] [d] [b] [c] IUMBV34,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 2 year (60% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [b] [c] IUMZICQ
0,31 Dec 25,4.83,4.42,3.97,3.78
1,30 Nov 25,4.91,4.49,4.06,3.91
2,31 Oct 25,5,4.57,4.20,4.03
3,30 Sep 25,5.03,4.58,4.19,4.03
4,31 Aug 25,5,4.55,4.10,3.93


,Date,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 5 year (95% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [a] [b] [c] [d] IUM5WTL,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 5 year (75% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [a] [b] [c] [d] IUMBV42,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 5 year (60% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [d] IUMZO27,Monthly interest rate of UK monetary financial institutions (excl. Central Bank) sterling 5 year (90% LTV) fixed rate mortgage to households (in percent) not seasonally adjusted [d] IUMZO28
0,31 Dec 25,4.76,4.00,3.88,4.35
1,30 Nov 25,4.82,4.07,3.96,4.41
2,31 Oct 25,4.89,4.17,4.10,4.50
3,30 Sep 25,4.9,4.16,4.07,4.51
4,31 Aug 25,4.86,4.11,4.03,4.43


,Date,Bank Rate
0,28-02-2003,3.75
1,03-03-2003,3.75
2,04-03-2003,3.75
3,05-03-2003,3.75
4,06-03-2003,3.75


,Title,CPIH ANNUAL RATE 00: ALL ITEMS 2015=100
0,CDID,L55O
1,Source dataset ID,MM23
2,PreUnit,NaN
3,Unit,%
4,Release date,21-01-2026
5,Next release,18 February 2026
6,Important notes,NaN
7,2003 MAR,1.6
8,2003 APR,1.5
9,2003 MAY,1.3


From this initial assessment, it's noticeable that some care has to be had with the column names, which for the two dataframes are too long. The proper data on the dataframe `cpih` only starts from row 7, with generic information on the first 6 rows, which can be safely removed.

In [2]:
# Renaming the necessary columns
fixed_2 = fixed_2.rename(columns = {fixed_2.columns[1]: '95_LTV_2y', 
                                    fixed_2.columns[2]: '90_LTV_2y',
                                    fixed_2.columns[3]: '75_LTV_2y',
                                    fixed_2.columns[4]: '60_LTV_2y'})
fixed_5 = fixed_5.rename(columns = {fixed_5.columns[1]: '95_LTV_5y',
                                    fixed_5.columns[2]: '75_LTV_5y',
                                    fixed_5.columns[3]: '60_LTV_5y',
                                    fixed_5.columns[4]: '90_LTV_5y'})
cpih = cpih.rename(columns = {cpih.columns[0]: 'Date', cpih.columns[1]: 'CPIH'})

# Removing the first 7 rows of cpih
cpih = cpih.drop(index = cpih.index[:7]).reset_index(drop = True)

display(fixed_2.head(3))
display(fixed_5.head(3))
display(base_rate.head(3))
display(cpih.head(3))

,Date,95_LTV_2y,90_LTV_2y,75_LTV_2y,60_LTV_2y
0,31 Dec 25,4.83,4.42,3.97,3.78
1,30 Nov 25,4.91,4.49,4.06,3.91
2,31 Oct 25,5,4.57,4.20,4.03


,Date,95_LTV_5y,75_LTV_5y,60_LTV_5y,90_LTV_5y
0,31 Dec 25,4.76,4.00,3.88,4.35
1,30 Nov 25,4.82,4.07,3.96,4.41
2,31 Oct 25,4.89,4.17,4.10,4.50


,Date,Bank Rate
0,28-02-2003,3.75
1,03-03-2003,3.75
2,04-03-2003,3.75


,Date,CPIH
0,2003 MAR,1.6
1,2003 APR,1.5
2,2003 MAY,1.3


Now that our dataframes are starting to gain shape, we'll proceed by running `info()` on the dataframes, as that lets us see if there are missing entries, and also if the column data types are aligned with what would be expected.

In [3]:
# Checking information on the dataframes
print('\nFixed 2 years:')
print(fixed_2.info())
print('\nFixed 5 years:')
print(fixed_5.info())
print('\nBase Rate:')
print(base_rate.info())
print('\nPCIH:')
print(cpih.info())


Fixed 2 years:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       276 non-null    object 
 1   95_LTV_2y  276 non-null    object 
 2   90_LTV_2y  212 non-null    object 
 3   75_LTV_2y  276 non-null    float64
 4   60_LTV_2y  168 non-null    float64
dtypes: float64(2), object(3)
memory usage: 10.9+ KB
None

Fixed 5 years:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       276 non-null    object 
 1   95_LTV_5y  276 non-null    object 
 2   75_LTV_5y  276 non-null    float64
 3   60_LTV_5y  83 non-null     float64
 4   90_LTV_5y  83 non-null     float64
dtypes: float64(3), object(2)
memory usage: 10.9+ KB
None

Base Rate:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 579

It seems that all of the `Date` columns are appearing as object `type`, so we;'ll need to convert to datetime objects, and also looking at the fixed-rates dataframes, some columns that were expected to be `float` are currently `objeect`, this means that there might be NaN or non-numeric values, so we'll need to verify. 

In [4]:
# Converting all columns to numeric and counting how many values are either NaN or non-numeric
rate_cols_2 = fixed_2.columns.drop('Date')
coerced_2 = fixed_2[rate_cols_2].apply(pd.to_numeric, errors="coerce")
missing_mask_2 = coerced_2.isna() & fixed_2.notna()
print('2 Year fixed-rate:\n', missing_mask_2.sum())

rate_cols_5 = fixed_5.columns.drop('Date')
coerced_5 = fixed_5[rate_cols_5].apply(pd.to_numeric, errors="coerce")
missing_mask_5 = coerced_5.isna() & fixed_5.notna()
print('5 Year fixed-rate:\n', missing_mask_5.sum())

# Joining it all back again
fixed_2[rate_cols_2] = coerced_2
fixed_5[rate_cols_5] = coerced_5

2 Year fixed-rate:
 60_LTV_2y     0.0
75_LTV_2y     0.0
90_LTV_2y     3.0
95_LTV_2y    65.0
Date          0.0
dtype: float64
5 Year fixed-rate:
 60_LTV_5y     0.0
75_LTV_5y     0.0
90_LTV_5y     0.0
95_LTV_5y    61.0
Date          0.0
dtype: float64


Numeric coercion was applied to all mortgage rate series.<br>
Values failing conversion while remaining present in the raw data were identified as placeholder entries denoting unavailable observations.<br>
These values were subsequently treated as missing during preprocessing.<br>
Among available LTV bands, both 60% and 75% LTV series were fully observed across 2-year and 5-year products.<br>
The 75% LTV series was selected as the primary modelling target due to its widespread use as a market benchmark and greater sensitivity to macroeconomic conditions.

The next step is then to clean the `Date` columns and format them the same way across all dataframes, this allows proper time-aware processing, indexing, and alignment across datasets. The CPIH series required a custom format due to "YYY MMM" strings.

In [5]:
# Converting all `Dates` to datetime
fixed_2['Date'] = fixed_2['Date'].apply(pd.to_datetime, dayfirst = True)
fixed_5['Date'] = fixed_5['Date'].apply(pd.to_datetime, dayfirst = True)
base_rate['Date'] = base_rate['Date'].apply(pd.to_datetime, dayfirst = True)
cpih['Date'] = cpih['Date'].apply(pd.to_datetime, format='%Y %b')

# Confirming changes applied as predicted
display(fixed_2.head(3))
print(fixed_2.dtypes)

,Date,95_LTV_2y,90_LTV_2y,75_LTV_2y,60_LTV_2y
0,2025-12-31,4.83,4.42,3.97,3.78
1,2025-11-30,4.91,4.49,4.06,3.91
2,2025-10-31,5.00,4.57,4.20,4.03


Date         datetime64[ns]
95_LTV_2y           float64
90_LTV_2y           float64
75_LTV_2y           float64
60_LTV_2y           float64
dtype: object


We can now group `base_rate` by month, and then remove all the days from the `Date` column, keeping the formatting the same across all dataframes.

In [6]:
# Grouping by Month
base_rate_monthly = base_rate.groupby(base_rate['Date'].dt.to_period('M').rename('Month')).mean().reset_index()

# Convert Period back to datetime at start of month
base_rate_monthly['Date'] = base_rate_monthly['Month'].dt.to_timestamp()
base_rate_monthly.drop(columns='Month', inplace=True)

# Dropping day from all dataframes
fixed_2['Date'] = fixed_2['Date'].dt.to_period('M').dt.to_timestamp()
fixed_5['Date'] = fixed_5['Date'].dt.to_period('M').dt.to_timestamp()
base_rate_monthly['Date'] = base_rate_monthly['Date'].dt.to_period('M').dt.to_timestamp()
cpih['Date'] = cpih['Date'].dt.to_period('M').dt.to_timestamp()


display(base_rate_monthly.head())
display(fixed_2.head())
display(fixed_5.head())
cpih.head()

,Date,Bank Rate
0,2003-02-01,3.75
1,2003-03-01,3.75
2,2003-04-01,3.75
3,2003-05-01,3.75
4,2003-06-01,3.75


,Date,95_LTV_2y,90_LTV_2y,75_LTV_2y,60_LTV_2y
0,2025-12-01,4.83,4.42,3.97,3.78
1,2025-11-01,4.91,4.49,4.06,3.91
2,2025-10-01,5.00,4.57,4.20,4.03
3,2025-09-01,5.03,4.58,4.19,4.03
4,2025-08-01,5.00,4.55,4.10,3.93


,Date,95_LTV_5y,75_LTV_5y,60_LTV_5y,90_LTV_5y
0,2025-12-01,4.76,4.00,3.88,4.35
1,2025-11-01,4.82,4.07,3.96,4.41
2,2025-10-01,4.89,4.17,4.10,4.50
3,2025-09-01,4.90,4.16,4.07,4.51
4,2025-08-01,4.86,4.11,4.03,4.43


,Date,CPIH
0,2003-03-01,1.6
1,2003-04-01,1.5
2,2003-05-01,1.3
3,2003-06-01,1.2
4,2003-07-01,1.3


The preprocessing for our datasets is almost complete, the last step is to join all the dataframes into a single one with all the information necessary.<br>
The datasets will be merged on month-start dates, retaining only the target series (75% LTV mortgage rate) and relevant macroeconomic features (Bank Rate and CPIH). This produces a single, time-aligned table for modeling.<br>
Two dataframes were created, so we can proceed with the analysis of both a 2 year and a 5 year fixed rate for mortgages with 75% LTV without risking information being leaked between the rates or compromising the predictions.

In [7]:
# Joining all the dataframes / Inner was selected to make sure the time period is the same across all columns
# 2 year fixed
df_2y = fixed_2[['Date', '75_LTV_2y']].merge(
    base_rate_monthly[['Date', 'Bank Rate']],
    on='Date',
    how='inner'
).merge(
    cpih[['Date', 'CPIH']],
    on='Date',
    how='inner'
)
# 5 year fixed
df_5y = fixed_5[['Date', '75_LTV_5y']].merge(
    base_rate_monthly[['Date', 'Bank Rate']],
    on='Date',
    how='inner'
).merge(
    cpih[['Date', 'CPIH']],
    on='Date',
    how='inner'
)

# Setting the `Date` as the index so resampling and plotting are easier 
df_2y.set_index('Date', inplace=True)
df_5y.set_index('Date', inplace=True)

# Confirming
print(df_2y.head())
print(df_5y.info())

            75_LTV_2y  Bank Rate CPIH
Date                                 
2025-12-01       3.97   3.904762  3.6
2025-11-01       4.06   4.000000  3.5
2025-10-01       4.20   4.000000  3.8
2025-09-01       4.19   4.000000  4.1
2025-08-01       4.10   4.050000  4.1
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2025-12-01 to 2003-03-01
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   75_LTV_5y  274 non-null    float64
 1   Bank Rate  274 non-null    float64
 2   CPIH       274 non-null    object 
dtypes: float64(2), object(1)
memory usage: 8.6+ KB
None


## 2. Feature Engineering 
We'll now proceed with creating lagged mortage rates so we can capture the persistance of the housing market in relation to current market trends, a spread feature to capture the lender pricing behaviour, short term rolling mean capture short-term trends, and finally create our feature and target variables. <br>
On this project, two target variables will be used, one for a fixed 2-year fixed rate with 75% LTV, and another for a fixed 5-year fixed rate with 75% LTV.

Train/Test Split – Use time-aware split to avoid future data leakage
Baseline Model: Linear Regression – Simple regression to establish a baseline
ML Model: Random Forest – More advanced model to capture non-linear trends
Model Evaluation – Compare predictions using metrics like MAE, RMSE
Model Saving & Inference Function – Save trained models and create a reusable prediction function
Conclusion

In [8]:
# Lagged Mortgage rates
# 2 year
df_2y['mortgage_lag_1'] = df_2y['75_LTV_2y'].shift(1)
df_2y['mortgage_lag_3'] = df_2y['75_LTV_2y'].shift(3)

# 5 year
df_5y['mortgage_lag_1'] = df_5y['75_LTV_5y'].shift(1)
df_5y['mortgage_lag_3'] = df_5y['75_LTV_5y'].shift(3)

# Mortgage spread
df_2y['spread_base_rate'] = df_2y['75_LTV_2y'] - df_2y['Bank Rate']
df_5y['spread_base_rate'] = df_5y['75_LTV_5y'] - df_5y['Bank Rate']

# Rolling mean
df_2y['mortgage_roll_3'] = df_2y['75_LTV_2y'].rolling(window=3).mean()
df_5y['mortgage_roll_3'] = df_5y['75_LTV_5y'].rolling(window=3).mean()

# Drop rows with NaNs created by the lagged rates
df_2y_fe = df_2y.dropna().copy()
df_5y_fe = df_5y.dropna().copy()

# Creating feature and target dataframes
# 2 year
X_2y = df_2y_fe[['Bank Rate', 'CPIH', 'mortgage_lag_1', 'mortgage_lag_3', 'mortgage_roll_3', 'spread_base_rate']]
y_2y = df_2y_fe['75_LTV_2y']

# 5 year
X_5y = df_5y_fe[['Bank Rate', 'CPIH', 'mortgage_lag_1', 'mortgage_lag_3', 'mortgage_roll_3', 'spread_base_rate']]
y_5y = df_5y_fe['75_LTV_5y']